In [ ]:
import tkinter as tk
from tkinter import messagebox
import os
import cv2
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy

In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter.ttk import Combobox
from tkinter import scrolledtext
import mysql.connector
import cv2
import os
import numpy as np
from PIL import Image

# -------------------- Create dataset folder if it doesn't exist -------------------- #
if not os.path.exists("dataset"):
    os.makedirs("dataset")

# -------------------- Database Connection -------------------- #
try:
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="authorized_user"
    )
    mycursor = mydb.cursor()
except mysql.connector.Error as e:
    messagebox.showerror("Database Error", f"{e}")

# -------------------------- Helper Functions -------------------------- #
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, pred = clf.predict(gray_img[y:y+h, x:x+w])
        confidence = int(100 * (1 - pred / 300))

        # Fetch name from database
        try:
            mycursor.execute("SELECT name FROM my_table WHERE id=%s", (id,))
            s = mycursor.fetchone()
            if s:
                name = s[0]
            else:
                name = "Unknown"
        except mysql.connector.Error as e:
            name = "Unknown"
            print(f"DB Error: {e}")

        # Display the name or unauthorized
        if confidence > 80:
            cv2.putText(img, str(name), (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1)
        else:
            cv2.putText(img, "UNAUTHORIZED", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1)

    return img

# -------------------------- Dataset Generation -------------------------- #
def generate_dataset():
    if t1.get() == "" or t2.get() == "" or t3.get() == "":
        messagebox.showerror('Error', 'Please provide Name, Age, and Address')
        return

    # Insert into MySQL
    try:
        sql = "INSERT INTO my_table(Name, Age, Address) VALUES (%s, %s, %s)"
        val = (t1.get(), int(t2.get()), t3.get())
        mycursor.execute(sql, val)
        mydb.commit()

        mycursor.execute("SELECT LAST_INSERT_ID()")
        id = mycursor.fetchone()[0]
        print(f"Inserted ID: {id}")

    except mysql.connector.Error as e:
        messagebox.showerror("Database Error", f"{e}")
        return
    except ValueError:
        messagebox.showerror("Error", "Age must be a number")
        return

    # OpenCV Haarcascade
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    if face_classifier.empty():
        messagebox.showerror('Error', 'Haarcascade XML file not found!')
        return

    def face_cropped(img):
        if img is None:
            return None
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]

    # Capture face images
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        messagebox.showerror('Error', 'Could not open webcam')
        return

    img_id = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        face = face_cropped(frame)
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"dataset/user.{id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face)

        if cv2.waitKey(1) == 13 or img_id == 50:  # Enter key or 50 images
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('Result', 'Dataset generation completed!')

# -------------------------- Training -------------------------- #
def train_classifier():
    data_dir = "dataset"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)

    if len(faces) == 0 or len(ids) == 0:
        messagebox.showerror("Error", "No images found. Generate dataset first!")
        return

    ids = np.array(ids)
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo('Result', 'Training dataset completed!')

# -------------------------- Face Detection -------------------------- #
def detect_face():
    faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    if faceCascade.empty():
        messagebox.showerror('Error', 'Haarcascade XML file not found!')
        return

    clf = cv2.face.LBPHFaceRecognizer_create()
    if not os.path.exists("classifier.xml"):
        messagebox.showerror('Error', 'Train the dataset first!')
        return
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        messagebox.showerror('Error', 'Could not open webcam')
        return

    while True:
        ret, img = video_capture.read()
        if not ret:
            print("Failed to grab frame from webcam.")
            break

        img = draw_boundary(img, faceCascade, 1.3, 5, (255, 255, 255), clf)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) == 13:  # Enter key
            break

    video_capture.release()
    cv2.destroyAllWindows()

# -------------------------- GUI Setup -------------------------- #
window = tk.Tk()
window.title("Face Recognition System")

# Labels and Entries
l1 = tk.Label(window, text="Name", font=("Algerian", 20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian", 20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian", 20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

# Buttons
b1 = tk.Button(window, text="Training", font=("Algerian", 20), bg="orange", fg="red", command=train_classifier)
b1.grid(column=0, row=4)

b2 = tk.Button(window, text="Detect Faces", font=("Algerian", 20), bg="green", fg="white", command=detect_face)
b2.grid(column=1, row=4)

b3 = tk.Button(window, text="Generate Dataset", font=("Algerian", 20), bg="pink", fg="black", command=generate_dataset)
b3.grid(column=2, row=4)

window.geometry("1000x250")
window.mainloop()
